In [1]:
from sqlalchemy import create_engine
import pandas as pd
import plotly.graph_objects as go
import shared_funcs


In [2]:
sel_cols = ['year', 'state', 'station_name', 'lat', 'lon', 'ttl_year']
convert_dict = {'year': int, 'state': 'string', 'station_name': 'string', 'ttl_year': int}


In [4]:
url = shared_funcs.database_path()
engine = create_engine(url=url)

with open('sql/analytics_mapping.sql', 'r') as sql_read:
    df = pd.read_sql(sql_read.read(), engine.connect())


In [17]:
select_year = 2023
df = df.astype(convert_dict).query('year==@select_year')
df = df[sel_cols].sort_values(by=['ttl_year'], ascending=True)
df.reset_index(drop=True, inplace=True)
df['text'] = df['station_name'] + '<br>PressureDays ' + (df['ttl_year']).astype(str)
print(df.dtypes)


year                     int64
state           string[python]
station_name    string[python]
lat                    float64
lon                    float64
ttl_year                 int64
text            string[python]
dtype: object


In [19]:
df[:120]


,year,state,station_name,lat,lon,ttl_year,text
0,2023,FL,JACKSONVILLE INTERNATIONAL AIRPORT,30.495,-81.694,0,JACKSONVILLE INTERNATIONAL AIRPORT<br>Pressure...
1,2023,CA,J. WAYNE APT-ORANGE CO APT,33.680,-117.867,0,J. WAYNE APT-ORANGE CO APT<br>PressureDays 0
2,2023,HI,LIHUE AIRPORT,21.980,-159.339,0,LIHUE AIRPORT<br>PressureDays 0
3,2023,FL,MIAMI INTERNATIONAL AIRPORT,25.788,-80.317,0,MIAMI INTERNATIONAL AIRPORT<br>PressureDays 0
4,2023,HI,HONOLULU INTERNATIONAL AIRPORT,21.324,-157.939,0,HONOLULU INTERNATIONAL AIRPORT<br>PressureDays 0
...,...,...,...,...,...,...,...
107,2023,NY,SYRACUSE HANCOCK INTERNATIONAL AP,43.111,-76.104,49,SYRACUSE HANCOCK INTERNATIONAL AP<br>PressureD...
108,2023,NY,BUFFALO NIAGARA INTERNATIONAL AP,42.940,-78.736,49,BUFFALO NIAGARA INTERNATIONAL AP<br>PressureDa...
109,2023,KS,WICHITA EISENHOWER NATIONAL,37.648,-97.430,50,WICHITA EISENHOWER NATIONAL<br>PressureDays 50
110,2023,CO,CITY OF COLORADO SPRINGS MUNICIPAL AP,38.809,-104.689,52,CITY OF COLORADO SPRINGS MUNICIPAL AP<br>Press...


In [20]:
limits = [(0,30),(30,60),(60,80),(80,100),(100,120)]
colors = ["olive", "green", "yellow", "blue", "red"]
cities = []

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['ttl_year'],
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = f'{select_year} US City Pressure Days<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()


In [21]:
file = f"{select_year}-US-City-Pressure-Days-bubblemap.pdf"
fig.write_image(file, scale=2)
